In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load dataset
df = pd.read_csv("Concrete_Data.csv")  # Ensure correct filename
X = df[['Cement']].values  # Selecting Cement as input feature
y = df[['Concrete_compressive_strength']].values  # Target variable

# Split data (60% training, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize input feature (recommended for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Define Neural Network class
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),  # Activation function
            nn.Linear(hidden_size, 1)  # Output layer (Regression)
        )

    def forward(self, x):
        return self.model(x)

# Train and evaluate models with different hidden layer sizes
hidden_layer_sizes = [8, 16, 32, 64]
best_model = None
best_rmse = float("inf")
training_losses = {}

for hidden_size in hidden_layer_sizes:
    model = NeuralNet(input_size=1, hidden_size=hidden_size)
    criterion = nn.MSELoss()  # Mean Squared Error Loss
    optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

    # Training loop
    epochs = 500
    loss_history = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())

    # Evaluate on validation data
    model.eval()
    with torch.no_grad():
        val_predictions = model(X_val)
        val_rmse = np.sqrt(mean_squared_error(y_val.numpy(), val_predictions.numpy()))

    print(f"Hidden Layer {hidden_size} - Validation RMSE: {val_rmse:.4f}")

    training_losses[hidden_size] = loss_history

    # Track best model
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_model = model

# Evaluate the best model on test data
best_model.eval()
with torch.no_grad():
    test_predictions = best_model(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test.numpy(), test_predictions.numpy()))

print(f"\nBest Model Test RMSE: {test_rmse:.4f}")

# Plot training loss for the best model
plt.figure(figsize=(8, 5))
plt.plot(training_losses[hidden_layer_sizes[0]], label=f"Hidden {hidden_layer_sizes[0]}")
plt.plot(training_losses[hidden_layer_sizes[1]], label=f"Hidden {hidden_layer_sizes[1]}")
plt.plot(training_losses[hidden_layer_sizes[2]], label=f"Hidden {hidden_layer_sizes[2]}")
plt.plot(training_losses[hidden_layer_sizes[3]], label=f"Hidden {hidden_layer_sizes[3]}")
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Training Loss vs Epochs')
plt.legend()
plt.show()


KeyError: "None of [Index(['Cement'], dtype='object')] are in the [columns]"